<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Проверим-дубликаты" data-toc-modified-id="Проверим-дубликаты-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Проверим дубликаты</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверим-значения" data-toc-modified-id="Проверим-значения-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Проверим значения</a></span><ul class="toc-item"><li><span><a href="#Удалим-столбцы:-которые-больше-не-потребуются" data-toc-modified-id="Удалим-столбцы:-которые-больше-не-потребуются-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Удалим столбцы: которые больше не потребуются</a></span></li><li><span><a href="#Результаты-проверки" data-toc-modified-id="Результаты-проверки-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Результаты проверки</a></span></li></ul></li><li><span><a href="#Обработаем-выбросы-и-границы-значений-в-упомянутых-выше-признаках" data-toc-modified-id="Обработаем-выбросы-и-границы-значений-в-упомянутых-выше-признаках-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обработаем выбросы и границы значений в упомянутых выше признаках</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li></ul></li><li><span><a href="#Разделим-и-закодируем-выборки" data-toc-modified-id="Разделим-и-закодируем-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделим и закодируем выборки</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Анализ моделей</a></span></li></ul></div>

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [27]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import matplotlib.pyplot as plt


#  Подготовка данных

In [26]:
frame = pd.read_csv("/datasets/autos.csv")

In [3]:
frame.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:

frame["VehicleType"] = frame["VehicleType"].fillna(frame["VehicleType"].mode().values[0])
frame["Gearbox"] = frame["Gearbox"].fillna(frame["Gearbox"].mode().values[0])
frame["Model"] = frame["Model"].fillna(frame["Model"].mode().values[0])
frame["FuelType"] = frame["FuelType"].fillna(frame["FuelType"].mode().values[0])

frame["NotRepaired"] = frame["NotRepaired"].fillna("no")

При очистке данных от пропусков стоял выбор между полным удалением объектов с пропусками(из за того что мы не могли на основе других параметров объекта достоверно восстановить упущенную информацию) и заполнение их наиболее часто встречающимся значением(так как это все категорийные признаки). По моим подсчетам, удаление объектов с пропусками привело бы к потере более 20% данных, что критично

Проверим, что значения категорийных признаков не дублируют друг друга 

In [6]:
objects_columns = ["VehicleType","Gearbox","Model","FuelType","Brand","NotRepaired"]
for column in objects_columns:
    print(column, pd.Series(frame[column].unique()).str.lower().duplicated().sum())

VehicleType 0
Gearbox 0
Model 0
FuelType 0
Brand 0
NotRepaired 0


## Проверим дубликаты 

In [7]:
frame.duplicated().sum()

5

In [8]:
frame = frame.drop_duplicates()
frame.duplicated().sum()

0

### Вывод

дубликатов больше нет

## Проверим значения

In [9]:
frame.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354364.000000,354364.000000,354364.000000,354364.000000,354364.000000,354364.0,354364.000000
mean,4416.655608,2004.234471,110.093723,128211.373051,5.714641,0.0,50508.461698
std,4514.159690,90.228593,189.851590,37905.136957,3.726433,0.0,25783.124276
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49409.500000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### Удалим столбцы: которые больше не потребуются

In [10]:
frame_with_date = frame.copy()
frame = frame.drop(["DateCrawled","DateCreated","LastSeen","NumberOfPictures","PostalCode"],axis = 1)

### Результаты проверки

Проанализируем значения числовых признаков перед применения к ним алгоритма очитки выбросов 
1. Можно заметить, что в столбце RegistrationYear присутствуют аномальные значения, предлагаю установить границы, в котором может находится значение этого столбца. А именно, год регистрации не может быть больше 2020, и не может быть меньше 1930. Все показания выше или ниже этих значений приведем к максимальной или минимальной границе __(признак не нуждается в поиске выбросов)__  

1. Можно заметить, что в столбце RegistrationMonth, встречается значение 0, которое корректнее было бы изменить на 1 __(признак не нуждается в поиске выбросов)__  

1. Так же в Power присутствуют аномальные значения (20 000 лошадиных сил), ограничим их мощностью БелАЗ(а), т.е. 3500 __(признак нуждается в поиске выбросов)__  

1. Столбец Kilometer имеет достаточно реальные значения, поэтому смысла искать там выбросы нет __(признак не нуждается в поиске выбросов)__  

1. Значения из столбца Price обладают высоким стандартным отклонениям, что пагубно будет влиять на работу алгоритма  __(признак нуждается в поиске выбросов)__  

1. Такие признаки как NumberOfPictures и PostalCode __(признаки не нуждается в поиске выбросов)__ , так как не влияют на реальную стоимость автомобиля (их мы просто удалим)



## Обработаем выбросы и границы значений в упомянутых выше признаках 

In [11]:
# RegistrationYear 
def Balance_RegistrationYear(value):
    if value > 2020:
        return 2020
    elif value < 1930:
        return 1930
    else:
        return value
frame["RegistrationYear"] = frame["RegistrationYear"].apply(Balance_RegistrationYear)
# RegistrationMonth
frame.loc[frame['RegistrationMonth'] == 0, 'RegistrationMonth'] = 1
# Power
frame.loc[frame['Power'] > 3500, 'Power'] = 3500

In [12]:
def remove_ouliers(frame,column):
    q25=np.array(frame[column].quantile(0.25))
    
    q75=np.array(frame[column].quantile(0.75))
    first_part=q25-1.5*(q75-q25)
    second_part=q75+1.5*(q75-q25)
    del_index = []
    for index_value, value in zip(frame[column].index,frame[column]):
        if second_part <= value or value <= first_part:
            del_index.append(index_value)
    
    print('Количество строк, выбранных для удаления ' + str(column)+":",len(del_index))
    return del_index

In [13]:
array_num_col = ["Price","Power"]
count = 0 
for column in array_num_col:
    index_del = remove_ouliers(frame,column)
    count += len(index_del)
    frame = frame.drop(index_del,axis = 0)
print("Было удалено:", count)

Количество строк, выбранных для удаления Price: 18979
Количество строк, выбранных для удаления Power: 5153
Было удалено: 24132


In [14]:
frame.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
Price,1.000000,0.365565,0.433664,-0.279190,0.108351
RegistrationYear,0.365565,1.000000,0.092088,-0.132255,0.042231
Power,0.433664,0.092088,1.000000,0.134619,0.125639
Kilometer,-0.279190,-0.132255,0.134619,1.000000,0.015735
RegistrationMonth,0.108351,0.042231,0.125639,0.015735,1.000000


### Выводы

- Пропуски были только в категорийных признаках, поэтому заполнения выполнял модой 
- Дубликатов внутри признаков найдено не было
- Дубликаты объектов: было найдено 5 штук - удалены
- Проверена корреляция: все в рамках разумных пределов, наиболее коррелирующим с целевим признаком является признак отвечающий за мощность 
- Данные были очищены от выбросов, суммарно было удалено 73 145 объектов, что составляет около 20%. Это достаточно большой процент, поэтому в случае низких результатов модели имеет смысл изменить(увеличить) коэффициент при межквартильном размахе, что уменьшит общий объем объектов идентифицирующих как выбросы (при коэффициенте равном 2 сумма количества удаленных объектов уменьшится ~20 000 (приблизительно 5% от изначальной выборки))

# Разделим и закодируем выборки

In [15]:
frame_ohe = frame.copy()
frame_ohe = pd.get_dummies(frame_ohe)
frame_light = frame.copy()
frame_light = frame_light.astype({"VehicleType":'category',
                                  "Gearbox":'category',
                                  "Model":'category',
                                  "FuelType":'category',
                                  "Brand":'category',
                                  "NotRepaired":'category'})

Разделим данные на выборки

In [16]:
trainX,testX,trainY,testY = train_test_split(frame_ohe.drop("Price",axis = 1),
                                             frame_ohe["Price"],
                                             test_size = 0.25,
                                             random_state = 42)

trainX_wo_ohe,testX_wo_ohe,trainY_wo_ohe,testY_wo_ohe = train_test_split(frame.drop("Price",axis = 1),
                                             frame["Price"],
                                             test_size = 0.25,
                                             random_state = 42)
(trainX_wo_ohe_light,
 testX_wo_ohe_light,
 trainY_wo_ohe_light,
 testY_wo_ohe_light) = train_test_split(frame_light.drop("Price",axis = 1),
                                             frame_light["Price"],
                                             test_size = 0.25,
                                             random_state = 42)

#  Обучение моделей

In [17]:
%%time
# catboost
catboost = CatBoostRegressor(loss_function='RMSE')
parameters_cat = {'depth':[5,10], 'learning_rate':np.arange(0.1,1,0.2)}
catboost_grid = catboost.grid_search(parameters_cat,
            Pool(trainX_wo_ohe, trainY_wo_ohe, cat_features=["VehicleType","Gearbox",
                                                             "Model","FuelType",
                                                             "Brand","NotRepaired"]),
            cv=3,
            verbose=True,
            plot=False)


bestTest = 1445.966066
bestIteration = 999

0:	loss: 1445.9660657	best: 1445.9660657 (0)	total: 32.4s	remaining: 4m 51s

bestTest = 1426.77088
bestIteration = 992

1:	loss: 1426.7708800	best: 1426.7708800 (1)	total: 1m 5s	remaining: 4m 21s

bestTest = 1423.815975
bestIteration = 995

2:	loss: 1423.8159746	best: 1423.8159746 (2)	total: 1m 37s	remaining: 3m 47s

bestTest = 1436.557374
bestIteration = 980

3:	loss: 1436.5573740	best: 1423.8159746 (2)	total: 2m 9s	remaining: 3m 14s

bestTest = 1445.892891
bestIteration = 727

4:	loss: 1445.8928905	best: 1423.8159746 (2)	total: 2m 41s	remaining: 2m 41s

bestTest = 1387.40679
bestIteration = 993

5:	loss: 1387.4067901	best: 1387.4067901 (5)	total: 3m 51s	remaining: 2m 34s

bestTest = 1394.650189
bestIteration = 670

6:	loss: 1394.6501885	best: 1387.4067901 (5)	total: 5m 2s	remaining: 2m 9s

bestTest = 1417.9342
bestIteration = 352

7:	loss: 1417.9341998	best: 1387.4067901 (5)	total: 6m 13s	remaining: 1m 33s

bestTest = 1445.073987
bestItera

In [18]:
catboost_grid["params"]


{'depth': 10, 'learning_rate': 0.1}

Проверим время обучения с найденными параметрами для каждой модели

In [19]:
%%time
final_cat = CatBoostRegressor(depth=catboost_grid["params"]["depth"], 
                              learning_rate=catboost_grid["params"]['learning_rate'],
                              loss_function='RMSE',verbose=100)
final_cat.fit(Pool(trainX_wo_ohe,trainY_wo_ohe,
                   cat_features=["VehicleType","Gearbox","Model","FuelType","Brand","NotRepaired"]))

0:	learn: 3199.3237813	total: 83.7ms	remaining: 1m 23s
100:	learn: 1460.6397495	total: 7.55s	remaining: 1m 7s
200:	learn: 1395.8414229	total: 14.7s	remaining: 58.6s
300:	learn: 1357.1510164	total: 22.4s	remaining: 52.1s
400:	learn: 1330.2085113	total: 30.4s	remaining: 45.4s
500:	learn: 1303.8544947	total: 38.2s	remaining: 38.1s
600:	learn: 1284.2563714	total: 46.3s	remaining: 30.7s
700:	learn: 1265.6215843	total: 54.3s	remaining: 23.2s
800:	learn: 1248.6235148	total: 1m 2s	remaining: 15.5s
900:	learn: 1233.3873270	total: 1m 10s	remaining: 7.73s
999:	learn: 1221.0471660	total: 1m 18s	remaining: 0us
CPU times: user 12min 6s, sys: 1min 6s, total: 13min 12s
Wall time: 1min 19s


In [20]:
%%time
# RandomForest
RF = RandomForestRegressor(criterion = "mse",random_state=42,n_estimators = 100,n_jobs=-1)
RF.fit(trainX,trainY)

CPU times: user 37min 47s, sys: 10.7 s, total: 37min 58s
Wall time: 3min 25s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [21]:
%%time
# LightGBM
lgb_train = lgb.Dataset(trainX_wo_ohe_light, trainY_wo_ohe_light)
lgb_test = lgb.Dataset(testX_wo_ohe_light, testY_wo_ohe_light, reference=lgb_train)
parameters_light = {'metric': 'l2', 'max_depth':10,"random_state": 42,"learning_rate":0.1}
light = lgb.train(parameters_light,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_train, lgb_test],
                verbose_eval=100)

[100]	training's l2: 2.04998e+06	valid_1's l2: 2.18059e+06
[200]	training's l2: 1.91936e+06	valid_1's l2: 2.10178e+06
[300]	training's l2: 1.84345e+06	valid_1's l2: 2.06858e+06
[400]	training's l2: 1.78335e+06	valid_1's l2: 2.04351e+06
[500]	training's l2: 1.73625e+06	valid_1's l2: 2.02733e+06
[600]	training's l2: 1.69256e+06	valid_1's l2: 2.01194e+06
[700]	training's l2: 1.65886e+06	valid_1's l2: 2.00455e+06
[800]	training's l2: 1.62604e+06	valid_1's l2: 1.99399e+06
[900]	training's l2: 1.59629e+06	valid_1's l2: 1.9874e+06
[1000]	training's l2: 1.56875e+06	valid_1's l2: 1.97986e+06
CPU times: user 52.9 s, sys: 717 ms, total: 53.7 s
Wall time: 4.68 s


Посчитаем ошибку для каждой модели и определим время предсказаний

In [22]:
%%time
print("CatBoost:",mean_squared_error(testY,final_cat.predict(testX_wo_ohe),squared=False))

CatBoost: 1410.0596136459287
CPU times: user 3.24 s, sys: 153 ms, total: 3.4 s
Wall time: 302 ms


In [23]:
%%time
print("RandomForest:",mean_squared_error(testY,RF.predict(testX),squared=False))

RandomForest: 1448.6789088096418
CPU times: user 10.9 s, sys: 2.77 s, total: 13.7 s
Wall time: 1.6 s


In [24]:
%%time
print("LightGBM:",(mean_squared_error(testY,light.predict(testX_wo_ohe_light),squared=False)))

LightGBM: 1407.0761519767168
CPU times: user 6.41 s, sys: 32.2 ms, total: 6.44 s
Wall time: 591 ms


#  Анализ моделей

Основываясь на полученные результаты можно сделать вывод, что наименьшую ошибку выдает CatBoost, на тренировку данной модели ушло ~1min 19s, в то же время на обучение рандомного леса ушло всего ~ 3min 25s, а на обучение LightGBM ушло 4.68 s с. Ошибки моделей не сильно отличаются, разница между крайними значениями составляет всего 3. В условии указано, что решающим факторами при выборе модели являются следующие показатели:
- Время обучения
- Время предсказания
- Качество предсказаний

Основываясь на данных признаков мы можем из удалить из сравнения RandomForest, так как его время обучения в разы больше.
Чтобы определить какая из двух оставшихся моделей нам подойдет больше нужно выбрать что для нас важнее, время обучение или ошибка. И там, и там разрыв между моделями небольшой. LightGBM обучается в несколько раз быстрее. Менее выгодно всего в данном случае смотреться CatBoost, эта модель проигрывает по 2 из 3 параметров:
- Время предсказания меньше (302 ms против 591 ms у LightGBM)
- Качество предсказаний ниже (1410 против 1407 у LightGBM)
- Время обучения больше (1min 19s против 4.68 s у LightGBM)

Основываясь на этом, можно сделать вывод, что лучше всего подойдет модель построенная на LightGBM 
